## Setup

In [1]:
import pandas as pd
import unicodedata
import json
import os

In [2]:
bairros_fortaleza = [
    "Aeroporto", "Aerolândia", "Alagadiço Novo", "Aldeota", "Alto da Balança",
    "Álvaro Weyne", "Amadeu Furtado", "Ancuri", "Antônio Bezerra", "Autran Nunes",
    "Barra do Ceará", "Barroso", "Bela Vista", "Benfica", "Bom Futuro",
    "Bom Jardim", "Bonsucesso", "Cais do Porto", "Cajazeiras", "Cambeba",
    "Canindezinho", "Carlito Pamplona", "Cidade 2000", "Cidade dos Funcionários", "Coaçu",
    "Cocó", "Conjunto Ceará I", "Conjunto Ceará II", "Conjunto Esperança", "Conjunto Palmeiras",
    "Couto Fernandes", "Cristo Redentor", "Curió", "Damas", "Demócrito Rocha",
    "Dendê", "Dias Macedo", "Dionísio Torres", "Dom Lustosa", "Edson Queiroz",
    "Ellery", "Engenheiro Luciano Cavalcante", "Farias Brito", "Fátima", "Floresta",
    "Genibaú", "Granja Lisboa", "Granja Portugal", "Guajeru", "Guararapes",
    "Henrique Jorge", "Itaoca", "Itaperi", "Jacarecanga", "Jangurussu",
    "Jardim América", "Jardim Cearense", "Jardim das Oliveiras", "Jardim Guanabara", "Jardim Iracema",
    "João XXIII", "Joaquim Távora", "Jóquei Clube", "José Bonifácio", "José de Alencar",
    "Lagoa Redonda", "Manuel Dias Branco", "Manoel Sátiro", "Maraponga", "Messejana",
    "Meireles", "Mondubim", "Monte Castelo", "Montese", "Moura Brasil",
    "Mucuripe", "Panamericano", "Papicu", "Parangaba", "Parque Araxá",
    "Parque Dois Irmãos", "Parque Iracema", "Parque Manibura", "Parquelândia", "Parreão",
    "Passaré", "Paupina", "Pedras", "Pici", "Pirambu",
    "Planalto Ayrton Senna", "Praia de Iracema", "Praia do Futuro I", "Praia do Futuro II", "Prefeito José Walter",
    "Presidente Kennedy", "Presidente Vargas", "Quintino Cunha", "Rodolfo Teófilo", "Sabiaguaba",
    "Salinas", "Santa Rosa", "São Bento", "São Gerardo", "São João do Tauape",
    "São José", "Sapiranga", "Serrinha", "Siqueira", "Varjota",
    "Vicente Pinzon", "Vila Ellery", "Vila Peri", "Vila União", "Vila Velha"
]

In [3]:
def normalizar_texto(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [4]:
bairros_normalizados = [normalizar_texto(bairro) for bairro in bairros_fortaleza]
pd.set_option('display.max_colwidth', None)

In [5]:
local_folder = "../../downloads/Portalnoticiasceara" 
dados = []

In [6]:
for root, dirs, files in os.walk(local_folder):
    for file in files:
        if file.endswith('.json'):  # Verificar se é um arquivo JSON
            file_path = os.path.join(root, file)
            print(f"Lendo o arquivo: {file_path}")

Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-25/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-26/metadata.json


## Recentes

In [7]:
for root, dirs, files in os.walk(local_folder):
    for file in files:
        if file.endswith('.json'):  # Verificar se é um arquivo JSON
            file_path = os.path.join(root, file)
            print(f"Lendo o arquivo: {file_path}")
            try:
                # Ler o conteúdo do arquivo JSON
                with open(file_path, 'r', encoding='utf-8') as json_file:
                    json_data = json.load(json_file)
                
                # Verificar se o conteúdo do JSON é uma lista
                if isinstance(json_data, list):
                    for item in json_data:
                        # Extrair informações relevantes
                        message = item.get('message', '')
                        message_id = item.get('id', None)
                        date = item.get('date', None)
                        media_path = item.get('media_path', None)
                        
                        if message:
                            # Normalizar a mensagem
                            message_normalizada = normalizar_texto(message)
                            
                            # Identificar bairros mencionados
                            bairros_mencionados = [
                                bairro_original for bairro_original, bairro_normalizado in zip(bairros_fortaleza, bairros_normalizados)
                                if bairro_normalizado in message_normalizada
                            ]
                            
                            # Se algum bairro foi identificado, salvar os dados
                            if bairros_mencionados:
                                for bairro in bairros_mencionados:
                                    dados.append({
                                        "id": message_id,
                                        "mensagem": message,
                                        "bairro": bairro,
                                        "data": date,
                                        "media_path": media_path
                                    })
            except Exception as e:
                print(f"Erro ao processar {file_path}: {e}")

# Salvar os dados extraídos em um novo arquivo JSON
output_file = './silver_metadata.json'
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(dados, json_file, ensure_ascii=False, indent=4)

print(f"Dados processados e salvos em {output_file}")

Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-25/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-26/metadata.json
Dados processados e salvos em ./silver_metadata.json


## Agrupado por Bairro

In [8]:
agrupado_por_bairro = {}

In [9]:
for root, dirs, files in os.walk(local_folder):
    for file in files:
        if file.endswith('.json'):  # Verificar se é um arquivo JSON
            file_path = os.path.join(root, file)
            print(f"Lendo o arquivo: {file_path}")
            try:
                # Ler o conteúdo do arquivo JSON
                with open(file_path, 'r', encoding='utf-8') as json_file:
                    json_data = json.load(json_file)
                
                # Verificar se o conteúdo do JSON é uma lista
                if isinstance(json_data, list):
                    for item in json_data:
                        # Extrair informações relevantes
                        message = item.get('message', '')
                        message_id = item.get('id', None)
                        date = item.get('date', None)
                        media_path = item.get('media_path', None)
                        
                        if message:
                            # Normalizar a mensagem
                            message_normalizada = normalizar_texto(message)
                            
                            # Identificar bairros mencionados
                            bairros_mencionados = [
                                bairro_original for bairro_original, bairro_normalizado in zip(bairros_fortaleza, bairros_normalizados)
                                if bairro_normalizado in message_normalizada
                            ]
                            
                            # Se algum bairro foi identificado, salvar os dados agrupados
                            for bairro in bairros_mencionados:
                                if bairro not in agrupado_por_bairro:
                                    agrupado_por_bairro[bairro] = []  # Inicializa a lista para o bairro
                                
                                # Verifica se a mensagem já está no agrupamento para evitar duplicação
                                mensagem_existente = any(
                                    mensagem['id'] == message_id and mensagem['mensagem'] == message 
                                    for mensagem in agrupado_por_bairro[bairro]
                                )
                                
                                if not mensagem_existente:
                                    agrupado_por_bairro[bairro].append({
                                        "id": message_id,
                                        "mensagem": message,
                                        "data": date,
                                        "media_path": media_path
                                    })
            except Exception as e:
                print(f"Erro ao processar {file_path}: {e}")

# Criar um arquivo JSON separado para cada bairro
output_folder = "./outputs"  # Pasta para salvar os arquivos
os.makedirs(output_folder, exist_ok=True)

for bairro, mensagens in agrupado_por_bairro.items():
    output_file = os.path.join(output_folder, f"{bairro}.json")
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(mensagens, json_file, ensure_ascii=False, indent=4)
    print(f"Arquivo criado para {bairro}: {output_file}")

Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-25/metadata.json
Lendo o arquivo: ../../downloads/Portalnoticiasceara/2024-12-26/metadata.json
Arquivo criado para Planalto Ayrton Senna: ./outputs/Planalto Ayrton Senna.json
Arquivo criado para Pici: ./outputs/Pici.json
Arquivo criado para Barra do Ceará: ./outputs/Barra do Ceará.json
Arquivo criado para Cristo Redentor: ./outputs/Cristo Redentor.json
Arquivo criado para Cajazeiras: ./outputs/Cajazeiras.json
Arquivo criado para Jangurussu: ./outputs/Jangurussu.json
Arquivo criado para Presidente Vargas: ./outputs/Presidente Vargas.json
Arquivo criado para Bom Jardim: ./outputs/Bom Jardim.json
Arquivo criado para Carlito Pamplona: ./outputs/Carlito Pamplona.json
Arquivo criado para Quintino Cunha: ./outputs/Quintino Cunha.json
Arquivo criado para Messejana: ./outputs/Messejana.json
